In [1]:
from utils.spark_session import get_spark_session
from pyspark.sql import SparkSession

spark: SparkSession = get_spark_session()

25/10/14 07:46:00 WARN Utils: Your hostname, DESKTOP-68EC7N1 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/14 07:46:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/dottier/venvs/flights_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dottier/.ivy2/cache
The jars for the packages stored in: /home/dottier/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8a36b16a-c8f6-4a44-9305-7996af5b0add;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 134ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   

In [4]:
spark.sql("SHOW TABLES in bronze").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze| airports|      false|
|   bronze|  flights|      false|
+---------+---------+-----------+



In [3]:
from delta.tables import DeltaTable

bronze_flights = DeltaTable.forName(spark, "bronze.flights")
bronze_flights.history().show(5)

25/10/12 17:28:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      1|2025-10-12 17:22:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|          0|  Serializable|        false|{numFiles -> 16, ...|        NULL|Apache-Spark/3.5....|
|      0|2025-10-12 16:13:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|       NULL|  Serializable|        false|{numFiles -> 158,...|        NULL|Apache-Spark/3.5....|
+-------+-

In [4]:
# Roll back the table to version 14
bronze_flights.restoreToVersion(0)


25/10/12 17:29:17 WARN DAGScheduler: Broadcasting large task binary with size 1076.9 KiB
25/10/12 17:29:19 WARN DAGScheduler: Broadcasting large task binary with size 1122.5 KiB
25/10/12 17:29:23 WARN DAGScheduler: Broadcasting large task binary with size 1159.9 KiB


DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [6]:
spark.sql("SHOW TABLES in silver").show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|   silver| dim_aircraft|      false|
|   silver|dim_aircrafts|      false|
|   silver| dim_airlines|      false|
|   silver| dim_airports|      false|
|   silver|  dim_runways|      false|
|   silver|  fct_flights|      false|
+---------+-------------+-----------+



In [7]:
spark.sql("DROP DATABASE silver CASCADE")

25/10/14 06:48:01 WARN TxnHandler: Cannot perform cleanup since metastore table does not exist


DataFrame[]

In [2]:
from utils.io_utils import read_df

silver_flights = read_df(spark, "silver", "fct_flights")
print(silver_flights.count())

silver_runways = read_df(spark, "silver", "dim_runways")
silver_runways.count()

25/10/14 07:46:16 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/14 07:46:16 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/10/14 07:46:18 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/10/14 07:46:18 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/10/14 07:46:18 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
25/10/14 07:46:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


183756


165

In [3]:
silver_runways.count()

165

In [ ]:
silver_runways.show(200)


+--------------------+--------------------+-----------+------------+--------+------------+---------+-----------+----------+------------------------+---------+-----------+--------------------+------------------+
|           runway_sk|          airport_sk|runway_name|true_heading| surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet| latitude|  longitude|effective_start_date|effective_end_date|
+--------------------+--------------------+-----------+------------+--------+------------+---------+-----------+----------+------------------------+---------+-----------+--------------------+------------------+
| 5018015408876227825| 4450828643872079020|         12|       117.7| Asphalt|        true|    false|    10285.0|     148.0|                     0.0|36.691074|  -4.507827|2025-10-14 06:49:...|              NULL|
| 1597118613431100831| 4450828643872079020|         13|       131.7| Asphalt|        true|    false|    10515.0|     148.0|                     0.0| 36.6845

In [6]:
import pyspark.sql.functions as sf

silver_runways.select(
    sf.count_distinct(sf.col("airport_sk"))
).show()

+--------------------------+
|count(DISTINCT airport_sk)|
+--------------------------+
|                        31|
+--------------------------+



In [7]:
a = silver_flights.dropDuplicates(["flight_composite_pk"])
a.count()

183756